In [1]:
### import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import os
from glob import glob

#np.random.seed(5)

Using TensorFlow backend.


In [49]:
train_dir = os.path.join('./data_origin/train/')
test_dir = os.path.join('./data_origin/test')

In [46]:
import os
from PIL import Image
imsidir = os.path.join('./data_origin/train/')
folder_list = os.listdir(imsidir)
print(folder_list)

['dyed-lifted-polyps', 'dyed-resection-margins', 'esophagitis', 'normal-cecum', 'normal-pylorus', 'normal-z-line', 'polyps', 'ulcerative-colitis']


In [69]:
import os
from PIL import Image
import cv2

folder_list = os.listdir(test_dir)
for foldername in folder_list:
    imsidir = os.path.join(test_dir, foldername)
    image_list = os.listdir(imsidir)
    for filename in image_list:
        imagename = os.path.join(imsidir,filename)
        img = Image.open(imagename)
        width , height = img.size
        if width == 1920:
            area = (340,0,width, height)
            cropped_img = img.crop(area)
            cropped_img.save(imagename);

In [70]:
#train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1)
train_datagen = ImageDataGenerator(
      rescale=1./255)      # 영상의 전체 픽셀을 255로 나눈다 - 영상이 0~1사이의 범위를 가지게 된다(normalization)
#      rotation_range=40,
 #     width_shift_range=0.2,
  #    height_shift_range=0.2,
   #   shear_range=0.2,
    #  zoom_range=0.2,
     # horizontal_flip=True,
      #fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(train_dir,
                                                   target_size=(90,72),
                                                   batch_size=100,
                                                   class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(test_dir,
                                                 target_size=(90,72),
                                                 batch_size=10,
                                                 class_mode='categorical')


Found 7200 images belonging to 8 classes.
Found 800 images belonging to 8 classes.


In [71]:
for data_batch, labels_batch in train_generator:
    print('배치 데이터 크기:', data_batch.shape)
    print('배치 레이블 크기:', labels_batch.shape)
    break

배치 데이터 크기: (100, 90, 72, 3)
배치 레이블 크기: (100, 8)


In [72]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3),
                activation='relu',
                input_shape=(90,72,3)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(8, activation='softmax'))

In [73]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 88, 70, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 86, 68, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 43, 34, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 93568)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               11976832  
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 1032      
Total params: 11,997,256
Trainable params: 11,997,256
Non-trainable params: 0
________________________________________________________________

In [74]:
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['acc'])

In [75]:
hist = model.fit_generator(train_generator,
                           steps_per_epoch=20,
                           epochs=10,
                           validation_data=test_generator,
                           validation_steps=100)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/10
20/20 [==============================] - 52s 3s/step - loss: 2.7301 - acc: 0.3490 - val_loss: 1.0986 - val_acc: 0.5220
Epoch 2/10
20/20 [==============================] - 33s 2s/step - loss: 0.9247 - acc: 0.5620 - val_loss: 0.7943 - val_acc: 0.6320
Epoch 3/10
20/20 [==============================] - 34s 2s/step - loss: 0.7844 - acc: 0.6290 - val_loss: 0.6981 - val_acc: 0.6900
Epoch 4/10
20/20 [==============================] - 32s 2s/step - loss: 0.7311 - acc: 0.6830 - val_loss: 0.6961 - val_acc: 0.6890
Epoch 5/10
20/20 [==============================] - 32s 2s/step - loss: 0.6523 - acc: 0.7045 - val_loss: 0.6401 - val_acc: 0.7180
Epoch 6/10
20/20 [==============================] - 32s 2s/step - loss: 0.6103 - acc: 0.7250 - val_loss: 0.5679 - val_acc: 0.7350
Epoch 7/10
20/20 [==============================] - 31s 2s/step - loss: 0.5868 - acc: 0.7530 - val_loss: 0.5666 - val_acc: 0.

In [76]:
scores = model.evaluate_generator(test_generator, steps=10)
print(scores)

[0.6474133551120758, 0.690000006556511]
